<a href="https://colab.research.google.com/github/zilioalberto/N3_PO/blob/main/N3_PO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##N3 -Pesquisa Operacional
##Professor Jaison
##Alunos:
#Alberto Zilio
#Roni Pereira


In [6]:
## Passo 01 - Bibliotecas do Projeto

# ---------------------------------------------
# Manipulação e análise de dados
# ---------------------------------------------
import pandas as pd   # Manipulação de dados em formato de tabela (DataFrames)
import numpy as np    # Operações numéricas, vetores, matrizes e números aleatórios

# Ajustes de visualização de DataFrames no notebook

pd.set_option("display.max_columns", 50)   # Máximo de colunas exibidas
pd.set_option("display.width", 120)        # Largura máxima da saída no console/notebook


# ---------------------------------------------
# Gráficos e visualização
# ---------------------------------------------

import matplotlib.pyplot as plt  # Biblioteca base para gráficos
import seaborn as sns            # Camada de alto nível para gráficos estatísticos

# Configurações gerais de gráficos

# Garante que os gráficos apareçam dentro do notebook
%matplotlib inline
sns.set_theme(style="whitegrid") # Define estilo visual padrão dos gráficos
plt.rcParams["figure.figsize"] = (10, 6)  # Tamanho padrão das figuras (largura x altura)


# ---------------------------------------------
# Estatística / distribuições (para Monte Carlo)
# ---------------------------------------------
from scipy import stats          # Funções estatísticas e distribuições de probabilidade


# ---------------------------------------------
# Utilitários diversos
# ---------------------------------------------
from pathlib import Path         # Facilita o trabalho com caminhos de arquivos e diretórios


# ---------------------------------------------
# Reprodutibilidade dos experimentos
# ---------------------------------------------
np.random.seed(42)               # Fixa a semente aleatória para reproduzir os resultados


In [9]:
## Passo 02 - Carregando os datasets a partir do GitHub

## Local de coleta do dataset:
# https://mavenanalytics.io/data-playground/pizza-place-sales

# ---------------------------------------------
# URL base do repositório no GitHub (modo "raw")
# ---------------------------------------------
base_url = "https://raw.githubusercontent.com/zilioalberto/N3_PO/main/pizza_sales/"

# Montagem das URLs completas de cada arquivo CSV
url_orders        = base_url + "orders.csv"
url_order_details = base_url + "order_details.csv"
url_pizzas        = base_url + "pizzas.csv"
url_pizza_types   = base_url + "pizza_types.csv"

print("URL orders       :", url_orders)
print("URL order_details:", url_order_details)
print("URL pizzas       :", url_pizzas)
print("URL pizza_types  :", url_pizza_types)

# ---------------------------------------------
# Leitura dos arquivos CSV a partir das URLs
# ---------------------------------------------

# Orders: informações de data/hora e ID dos pedidos
orders = pd.read_csv(url_orders)

# Order details: quais pizzas foram vendidas em cada pedido
order_details = pd.read_csv(url_order_details)

# Pizzas: informações de preço, tamanho e tipo de pizza
pizzas = pd.read_csv(url_pizzas)

# Pizza types: descrição, categoria e ingredientes (normalmente tem acentos)
pizza_types = pd.read_csv(url_pizza_types, encoding="latin1")

# ---------------------------------------------
# Visão geral inicial dos dados
# ---------------------------------------------
print("orders:")
display(orders.head())

print("\norder_details:")
display(order_details.head())

print("\npizzas:")
display(pizzas.head())

print("\npizza_types:")
display(pizza_types.head())


URL orders       : https://raw.githubusercontent.com/zilioalberto/N3_PO/main/pizza_sales/orders.csv
URL order_details: https://raw.githubusercontent.com/zilioalberto/N3_PO/main/pizza_sales/order_details.csv
URL pizzas       : https://raw.githubusercontent.com/zilioalberto/N3_PO/main/pizza_sales/pizzas.csv
URL pizza_types  : https://raw.githubusercontent.com/zilioalberto/N3_PO/main/pizza_sales/pizza_types.csv
orders:


,order_id,date,time
0,1,2015-01-01,11:38:36
1,2,2015-01-01,11:57:40
2,3,2015-01-01,12:12:28
3,4,2015-01-01,12:16:31
4,5,2015-01-01,12:21:30



order_details:


,order_details_id,order_id,pizza_id,quantity
0,1,1,hawaiian_m,1
1,2,2,classic_dlx_m,1
2,3,2,five_cheese_l,1
3,4,2,ital_supr_l,1
4,5,2,mexicana_m,1



pizzas:


,pizza_id,pizza_type_id,size,price
0,bbq_ckn_s,bbq_ckn,S,12.75
1,bbq_ckn_m,bbq_ckn,M,16.75
2,bbq_ckn_l,bbq_ckn,L,20.75
3,cali_ckn_s,cali_ckn,S,12.75
4,cali_ckn_m,cali_ckn,M,16.75



pizza_types:


,pizza_type_id,name,category,ingredients
0,bbq_ckn,The Barbecue Chicken Pizza,Chicken,"Barbecued Chicken, Red Peppers, Green Peppers,..."
1,cali_ckn,The California Chicken Pizza,Chicken,"Chicken, Artichoke, Spinach, Garlic, Jalapeno ..."
2,ckn_alfredo,The Chicken Alfredo Pizza,Chicken,"Chicken, Red Onions, Red Peppers, Mushrooms, A..."
3,ckn_pesto,The Chicken Pesto Pizza,Chicken,"Chicken, Tomatoes, Red Peppers, Spinach, Garli..."
4,southw_ckn,The Southwest Chicken Pizza,Chicken,"Chicken, Tomatoes, Red Peppers, Red Onions, Ja..."


In [ ]:
## Passo 03 - Tratamento básico e geração dos arquivos agregados

# 1) Converter datahora para datetime
df["datahora"] = pd.to_datetime(df["datahora"])

# 2) Garantir que valorMedida é numérico
df["valorMedida"] = pd.to_numeric(df["valorMedida"], errors="coerce")

# 3) Remover colunas que não serão usadas
colunas_remover = ["latitude", "longitude", "municipio", "uf", "codEstacao", "codigoestacao"]
df = df.drop(columns=colunas_remover, errors="ignore")

print("Colunas após remoção:")
print(df.columns)

# -------------------------------
# ARQUIVO 1: SOMA POR HORA CHEIA E ESTAÇÃO
# -------------------------------

# 4) Criar uma coluna com a hora inteira (suprimindo minutos/segundos)
df["datahora_hora"] = df["datahora"].dt.floor("H")

# 5) Agregar: somatório de valorMedida por hora cheia **e por estação**
dados_horarios = (
    df.groupby(["datahora_hora", "nomeEstacao"])["valorMedida"]
      .sum()
      .reset_index()
      .rename(columns={"datahora_hora": "datahora"})
)

# 6) Salvar arquivo horário
dados_horarios.to_csv("totaldata_horario_por_estacao.csv", index=False)

print("\nArquivo 1 (horário por estação) criado: totaldata_horario_por_estacao.csv")
print("Preview:")
print(dados_horarios.head())

# -----------------------------
# ARQUIVO 2: SOMA POR DIA E ESTAÇÃO
# -----------------------------

# 7) Criar coluna apenas com a data (sem hora)
df["data"] = df["datahora"].dt.date

# 8) Agregar: somatório de valorMedida por dia **e por estação**
dados_diarios = (
    df.groupby(["data", "nomeEstacao"])["valorMedida"]
      .sum()
      .reset_index()
)

# Converter coluna data de volta para datetime
dados_diarios["data"] = pd.to_datetime(dados_diarios["data"])

# 9) Salvar arquivo diário por estação
dados_diarios.to_csv("totaldata_diario_por_estacao.csv", index=False)

print("\nArquivo 2 (diário por estação) criado: totaldata_diario_por_estacao.csv")
print("Preview:")
print(dados_diarios.head())

# -----------------------------
# ARQUIVO 3: SÉRIE DIÁRIA DA CIDADE (SOMA, MÉDIA E Nº DE ESTAÇÕES)
# -----------------------------

chuva_diaria_cidade = (
    dados_diarios
    .groupby("data")["valorMedida"]
    .agg(
        soma_cidade="sum",      # soma da chuva em todas as estações nesse dia
        media_estacoes="mean",  # média da chuva entre as estações nesse dia
        n_estacoes="count"      # número de estações com dado nesse dia
    )
    .reset_index()
)

# 11) Salvar arquivo diário médio da cidade
chuva_diaria_cidade.to_csv("totaldata_diario_media_cidade.csv", index=False)

print("\nArquivo 3 (diário - cidade) criado: totaldata_diario_media_cidade.csv")
print("Preview:")
print(chuva_diaria_cidade.head())


In [ ]:
## Passo 04 - Análise exploratória da chuva diária (média da cidade)

# 1) Ler o arquivo diário da cidade (gerado no passo 03)
df_cidade = pd.read_csv("totaldata_diario_media_cidade.csv", parse_dates=["data"])

# 2) Ordenar por data e definir o índice
df_cidade = df_cidade.sort_values("data").set_index("data")

# 3) Opcional: renomear a coluna de média para ficar mais autoexplicativa
df_cidade = df_cidade.rename(columns={"media_estacoes": "chuva_media_mm"})

print("Visão geral do dataframe:")
print(df_cidade.info(), "\n")

print("Primeiras linhas:")
display(df_cidade.head())

print("\nResumo estatístico da chuva diária média (mm):")
display(df_cidade["chuva_media_mm"].describe())

print("\nResumo estatístico da soma diária da cidade (mm):")
display(df_cidade["soma_cidade"].describe())

print("\nNúmero de estações por dia (n_estacoes):")
display(df_cidade["n_estacoes"].value_counts().sort_index())


In [ ]:
## 4.1 – Gráfico da série temporal (chuva ao longo do ano)

fig, axes = plt.subplots(2, 1, figsize=(12, 6), sharex=True)

# Gráfico 1 – média
df_cidade["chuva_media_mm"].plot(ax=axes[0])
axes[0].set_title("Chuva diária média em Joinville (mm)")
axes[0].set_ylabel("Média (mm)")
axes[0].grid(True, linestyle="--", alpha=0.5)

# Gráfico 2 – soma
df_cidade["soma_cidade"].plot(ax=axes[1])
axes[1].set_title("Chuva diária total em Joinville (mm)")
axes[1].set_xlabel("Data")
axes[1].set_ylabel("Soma (mm)")
axes[1].grid(True, linestyle="--", alpha=0.5)

plt.tight_layout()
plt.show()



In [ ]:
## 4.2 – Histograma da chuva diária (todos os dias)

plt.figure(figsize=(8, 4))
sns.histplot(df_cidade["chuva_media_mm"], bins=30, kde=True)

plt.title("Distribuição da chuva diária média (mm) - Todos os dias")
plt.xlabel("Chuva diária média (mm)")
plt.ylabel("Frequência")

plt.tight_layout()
plt.show()



In [ ]:
## 4.3 – Histograma da chuva diária (apenas dias com chuva > 0 mm)

chuva_pos = df_cidade["chuva_media_mm"][df_cidade["chuva_media_mm"] > 0]

plt.figure(figsize=(8, 4))
sns.histplot(chuva_pos, bins=30, kde=True)

plt.title("Distribuição da chuva diária média (mm) - Dias com chuva > 0")
plt.xlabel("Chuva diária média (mm)")
plt.ylabel("Frequência")

plt.tight_layout()
plt.show()

print("Resumo apenas dos dias com chuva > 0 mm:")
display(chuva_pos.describe())




In [ ]:
## 4.4 - Distribuição da chuva diária média por mês

df_cidade_mes = df_cidade.copy()

# Extrair o número do mês a partir do índice (que é datetime)
df_cidade_mes["mes"] = df_cidade_mes.index.month

# Nomes dos meses (em português)
mes_labels = ["Jan", "Fev", "Mar", "Abr", "Mai", "Jun",
              "Jul", "Ago", "Set", "Out", "Nov", "Dez"]

plt.figure(figsize=(10, 5))

# order garante que os meses aparecem de 1 a 12 na ordem correta
sns.boxplot(
    data=df_cidade_mes,
    x="mes",
    y="chuva_media_mm",
    order=range(1, 13)
)

plt.title("Distribuição da chuva diária média por mês")
plt.xlabel("Mês")
plt.ylabel("Chuva diária média (mm)")

# Ajustar os rótulos do eixo X para nomes dos meses
plt.xticks(ticks=range(12), labels=mes_labels)

plt.tight_layout()
plt.show()


In [ ]:
## Passo 06 - Comparação entre bairros (estações)

# 5.1 - Carregar o arquivo diário por estação
df_estacoes = pd.read_csv("totaldata_diario_por_estacao.csv", parse_dates=["data"])

print("Visão geral da base diária por estação:")
print(df_estacoes.info(), "\n")

print("Primeiras linhas:")
display(df_estacoes.head())

print("\nEstações (bairros) disponíveis:")
print(df_estacoes["nomeEstacao"].unique())


In [ ]:
## 5.2 - Estatísticas de chuva por estação

estat_estacoes = (
    df_estacoes
    .groupby("nomeEstacao")["valorMedida"]
    .agg(
        total_chuva_mm = "sum",
        media_diaria_mm = "mean",
        max_diaria_mm = "max",
        dias_com_chuva = lambda x: (x > 0).sum()
    )
    .reset_index()
)

# Ordenar pela chuva total (do maior para o menor)
estat_estacoes = estat_estacoes.sort_values("total_chuva_mm", ascending=False)

print("Estatísticas por estação (bairros):")
display(estat_estacoes)
